In [1]:
import os 
import os.path
import copy
import nltk
import numpy as np

# clean content
def clean_content(imdb_str):
    
    imdb_str = imdb_str + ' '
    imdb_word_split = []
    count_flag = 0
    imdb_str = imdb_str.lower() # 大小写转换
    for i in range(len(imdb_str) - 1):

        if(imdb_str[i] is ' ')and(imdb_str[i+1] is not ' ')and(count_flag is 0):
            start_index = i+1
            count_flag = 1

        if(imdb_str[i] is not ' ')and(imdb_str[i+1] is ' ')and(count_flag is 1):
            end_index = i+1
            imdb_word_split.append(imdb_str[start_index:end_index])
            count_flag = 0
    return imdb_word_split

neg_file_name = 'rt-polarity.neg'
neg_file      = []
with open(neg_file_name,'r') as f:  #注意open 和 codecs.opem
    lines = f.readlines()
    for i in range(len(lines)):
        lines_clean = lines[i].rstrip('\n')
        neg_file.append(' ' + lines_clean)
            
pos_file_name = 'rt-polarity.pos'
pos_file      = []
with open(pos_file_name,'r') as f:  #注意open 和 codecs.opem
    lines = f.readlines()
    for i in range(len(lines)):
        lines_clean = lines[i].rstrip('\n')
        pos_file.append(' ' + lines_clean)  
        
print 'read_finish'

mr_train_file  = neg_file[500:5300] + pos_file[500:5300]
mr_train_label = np.concatenate([np.zeros(4800), np.ones(4800)], axis=0)
mr_test_file   = neg_file[0:500]  + pos_file[0:500]
mr_test_label  = np.concatenate([np.zeros(500), np.ones(500)], axis=0)

mr_train_split = []
for i in range(len(mr_train_label)):
    mr_train_split.append(clean_content(mr_train_file[i]))

mr_test_split = []
for i in range(len(mr_test_label)):
    mr_test_split.append(clean_content(mr_test_file[i]))

# calculate vab
mr_vab_count = {}

for i in range(len(mr_train_split)):
    mr_str = mr_train_split[i]
    for word_index in range(len(mr_str)):
        if (not mr_vab_count.has_key(mr_str[word_index])):
            mr_vab_count.update({mr_str[word_index]: 1})
        else:
            mr_vab_count[mr_str[word_index]] = mr_vab_count[mr_str[word_index]] + 1 


del_stop_word = False
if del_stop_word is True:
    # stop word list
    stop_word_list = []
    with open('/home/xiaosucheng/Desktop/IMDB/IMDB_Read_Data/imdb_stop_word.txt','r') as f:  #注意open 和 codecs.opem
        line = f.readlines()
        for i in range(len(line)):
            stop_word_list.append(line[i].rstrip('\n'))

    for i in range(len(stop_word_list)):
        if mr_vab_count.has_key(stop_word_list[i]):
            mr_vab_count.pop(stop_word_list[i])
    print 'delete stop word' 

# sort vab
mr_vab                  = {}
mr_vab_list_orgin       = mr_vab_count.keys()  # list
mr_vab_count_list_orgin = mr_vab_count.values() # list
mr_vab_list             = []
mr_vab_count_list       = []

mr_vab_index      = np.argsort(-np.array(mr_vab_count_list_orgin))  # np.argsort(-x) 降序排序
for i in range(len(mr_vab_list_orgin)):
    mr_vab.update({mr_vab_list_orgin[mr_vab_index[i]]: i + 1})    # 0 给stop_word　所以+1
    mr_vab_list.append(mr_vab_list_orgin[mr_vab_index[i]])
    mr_vab_count_list.append(mr_vab_count_list_orgin[mr_vab_index[i]])

print 'calculate vocabulary finished'            
            
    
vab_size = 20000
# word2index 如果不含有该词或者index超过vab_size,则置成０
mr_train_list_index = copy.deepcopy(mr_train_split)
for i in range(len(mr_train_split)):
    
    mr_str = mr_train_split[i]
    for word_index in range(len(mr_str)):
        if (not mr_vab.has_key(mr_str[word_index])) or (mr_vab[mr_str[word_index]] >= vab_size):
            mr_train_list_index[i][word_index] = 0
        else:
            mr_train_list_index[i][word_index] = mr_vab[mr_str[word_index]]

print 'train word2index finished'

mr_test_list_index = copy.deepcopy(mr_test_split)
for i in range(len(mr_test_split)):
    mr_str = mr_test_split[i]
    for word_index in range(len(mr_str)):
        if (not mr_vab.has_key(mr_str[word_index])) or (mr_vab[mr_str[word_index]] >= vab_size):
            mr_test_list_index[i][word_index] = 0
        else:
            mr_test_list_index[i][word_index] = mr_vab[mr_str[word_index]]
print 'test word2index finished'
    
    
import cPickle
MR = {}
MR['Vocabulary']       = mr_vab_list
MR['Vab_count']        = mr_vab_count_list
MR['Vab_Size']         = vab_size
MR['Label']            = ['neg','pos']
MR['Train_Origin']     = mr_train_file
MR['Train_Label']      = mr_train_label
MR['Train_Word_Split'] = mr_train_split
MR['Train_Word2Index'] = mr_train_list_index
MR['Test_Origin']      = mr_test_file
MR['Test_Label']       = mr_test_label
MR['Test_Word_Split']  = mr_test_split
MR['Test_Word2Index']  = mr_test_list_index
cPickle.dump(MR, open('./MR_ALL_20k.pkl','wb'))
print 'save finished'   

read_finish
calculate vocabulary finished
train word2index finished
test word2index finished
save finished


In [4]:
import numpy as np
import scipy.io as sio

# ELEC['Train_BOW'] = np.zeros([30000, 25000])
# for i in range(len(ELEC['Train_Word2Index'])):
    
#     elec_str = ELEC['Train_Word2Index'][i]
#     for j in range(len(elec_str)):
        
#         ELEC['Train_BOW'][elec_str[j],i] = ELEC['Train_BOW'][elec_str[j],i] + 1
        
# ELEC['Test_BOW'] = np.zeros([30000, 25000])
# for i in range(len(ELEC['Test_Word2Index'])):
    
#     elec_str = ELEC['Test_Word2Index'][i]
#     for j in range(len(elec_str)):
        
#         ELEC['Test_BOW'][elec_str[j],i] = ELEC['Test_BOW'][elec_str[j],i] + 1        
        
    
sio.savemat('MR_ALL_20k.mat',{ 'MR_Train_Word2Index': MR['Train_Word2Index'],
                               'MR_Vab': MR['Vocabulary'],
                               'MR_Test_Word2Index':MR['Test_Word2Index'],
                               'MR_Train_Label': list(MR['Train_Label']),
                               'MR_Test_Label': list(MR['Test_Label'])
                               })

print 'save mat finished'

save mat finished
